In [1]:
from sklearn.preprocessing import MinMaxScaler
import itertools
import pandas as pd
import torch

# get our predefined functions (for data processing, etc.)
import QCircNet.utils as ut

# get our predefined quantum circuits and neural networks
import QCircNet.circuits.CustomCircuits as cn
import QCircNet.QuantumNN as qnn


EXPERIMENT_ID = 69

DATA_PATH = "data/"


N_QUBITS = 4
FEAT_PER_QUBIT = 4

SUBSET = 100

EPOCHS = 16
MODEL_PATH = f"results/models_10/"
RESULTS_PATH = f"results/model_10_results_expID_{EXPERIMENT_ID}.csv"
RESULTS_DF = pd.DataFrame()

# for printing colored text output during training
RED_TEXT = "\033[91m"
GREEN_TEXT = "\033[92m"
RESET_COLOR = "\033[0m" 


In [2]:
# load the data
X_train, y_train, X_test, y_test  = ut.load_MNIST_data(DATA_PATH, subset=SUBSET)

# check shapes
print("train:\t", X_train.shape, y_train.shape)
print("test:\t", X_test.shape, y_test.shape)

train:	 torch.Size([100, 16]) torch.Size([100, 10])
test:	 torch.Size([100, 16]) torch.Size([100, 10])


In [3]:
cn.get_custom_circuits()

[QCircNet.circuits.CustomCircuits.Anusha2024Circuit,
 QCircNet.circuits.CustomCircuits.DoubleEntanglementVLCircuit,
 QCircNet.circuits.CustomCircuits.Ranga2024Circuit,
 QCircNet.circuits.CustomCircuits.Senokosov2024Circuit]

In [4]:
circuits = [
    cn.Ranga2024Circuit,
    # cn.Senokosov2024Circuit,
    # cn.Anusha2024Circuit,
    # cn.DoubleEntanglementVLCircuit,
]

runs = list(range(1, 11)) # runs are also used as seeds (reproducibility)
learning_rates = [0.01, 0.001]

In [ ]:
### RUN EXPERIMENT

id = 0
for seed, circuit, lr in itertools.product(runs, circuits, learning_rates):
    print(f"""{RED_TEXT}RUN {id+1}:{RESET_COLOR}\n {circuit.__name__} \n lr: {lr}""")

    # init the model
    model = qnn.QuantumClassifierMNIST(n_qubits=N_QUBITS, features_per_qubit=FEAT_PER_QUBIT, circuit=circuit, seed=seed)

    # train the model (predifined optimizer, etc.)
    trained_model, loss_history = ut.train_model(
        model, X_train, y_train,
        epochs=EPOCHS, batch_size=32, lr=lr, binary=True
    )
    
    # save trained model
    model_path = MODEL_PATH + f"model_expID_{EXPERIMENT_ID}_dfID_{id}.pth"
    torch.save(trained_model, model_path)

    # evaluate
    cross_entropy, accuracy, precision, recall, f1 = trained_model.evaluate(X_test, y_test)

    results_dict = {
        "id": id,
        "f1_score": f1, "precision": precision, "recall": recall, "accuracy": accuracy, "cross_entropy": cross_entropy,
        "seed": seed,
        "circuit": circuit.__name__,
        "learning_rate": lr,
        "loss_history": loss_history,
        # "scaler" : str(SCALER).replace("()", "")
    }
    print(f"\n{GREEN_TEXT}RESULTS:\n F1-Score:{f1}, Precision:{precision}, Recall:{recall}, Accuracy:{accuracy}, Loss:{cross_entropy} {RESET_COLOR} \n\n")
    
    # append the results_dict to the RESULTS_DF
    RESULTS_DF = pd.concat([RESULTS_DF, pd.DataFrame([results_dict])], ignore_index=True)
    RESULTS_DF.to_csv(path_or_buf=RESULTS_PATH) # save for each run (in case of a crash)

    id += 1

RUN 1:
 Ranga2024Circuit 
 lr: 0.01


Training progress:   0%|          | 0/3 [00:00<?, ?batch/s]

In [ ]:
RESULTS_DF

,id,f1_score,precision,recall,accuracy,cross_entropy,seed,circuit,learning_rate,loss_history
0,0,0.0,0.0,0.0,0.0,2.251537,1,Ranga2024Circuit,0.010,[0.2854444682598114]
1,1,0.0,0.0,0.0,0.0,2.256426,1,Ranga2024Circuit,0.001,[0.2854444682598114]
2,2,0.0,0.0,0.0,0.0,2.329711,2,Ranga2024Circuit,0.010,[0.36145487427711487]
3,3,0.0,0.0,0.0,0.0,2.331126,2,Ranga2024Circuit,0.001,[0.36145487427711487]
4,4,0.0,0.0,0.0,0.0,2.331426,3,Ranga2024Circuit,0.010,[0.36288121342658997]
5,5,0.0,0.0,0.0,0.0,2.332796,3,Ranga2024Circuit,0.001,[0.36288121342658997]
6,6,0.0,0.0,0.0,0.0,2.357646,4,Ranga2024Circuit,0.010,[0.4133490025997162]
7,7,0.0,0.0,0.0,0.0,2.358768,4,Ranga2024Circuit,0.001,[0.4133490025997162]
8,8,0.0,0.0,0.0,0.0,2.301875,5,Ranga2024Circuit,0.010,[0.3252934217453003]
9,9,0.0,0.0,0.0,0.0,2.304034,5,Ranga2024Circuit,0.001,[0.3252934217453003]


In [ ]:
# circuit = qc.QuantumCircuit

# # initialize the model
# model = qnn.QuantumNeuralNetwork(n_qubits=N_QUBITS, features_per_qubit=FEAT_PER_QUBIT, circuit=circuit, seed=1)

# # train the model (predifined optimizer, etc.)
# trained_model, loss_history, val_loss_history = ut.train_model(
#     model, X_train, y_train, epochs=EPOCHS, batch_size=64, lr=0.01
# ) #X_val, y_val,


# # Evaluate the model
# bce, accuracy, precision, recall, f1 = trained_model.evaluate(X_test, y_test)
# print(f"Binary Cross-Entropy on test data: {bce:.4f}")
# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")

In [ ]:
# # To load later
# loaded_model = torch.load("simple_model.pth")